In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
import numpy as np
import pandas as pd
import pandas_ta as ta
from ta import add_all_ta_features
from ta.utils import dropna
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from quiz.models import Pattern, HistoricalPatternActionPoint
from bourse_refs_api.models import Stock, StockHistory

In [32]:
queryset = list(StockHistory.objects.all().values())

In [34]:
df = pd.DataFrame(queryset)


In [35]:
df

,id,stock_id,date,max_price,min_price,last_price,last_deal_price,first_price,yesterday_price,value,volume,count
0,74501,5305844922895340,2019-08-14,4599,4450,4541,4578,4500,4432,10550901534,2323693,610
1,74502,5317427172344706,2019-08-14,21415,21415,21415,21415,21415,20396,3700426340,172796,1662
2,74503,5427792638736934,2019-08-14,17527,17105,17472,17527,17480,16693,48179415237,2757502,1017
3,74504,5516102131364383,2019-08-14,9779,9311,9722,9779,9575,9314,27385698321,2816800,738
4,74505,5564768007356822,2019-08-14,2749,2654,2686,2667,2700,2715,67545802763,25148785,3302
...,...,...,...,...,...,...,...,...,...,...,...,...
426144,74496,1625149423498289,2019-08-14,20690,19222,20156,20288,20000,20031,54268416506,2692459,1123
426145,74497,2135047529277416,2019-08-14,2797,2797,2797,2797,2797,2664,136392908,48764,9
426146,74498,3492952121304423,2019-08-14,10500,10000,10077,10100,10500,10303,10932147697,1084815,511
426147,74499,4247709727327181,2019-08-14,6940,6601,6798,6805,6690,6695,13082889460,1924382,427


In [38]:
df.stock_id = list(map(lambda x: str(x['stock_id']), queryset))

In [39]:
set(df.stock_id

0         5305844922895340
1         5317427172344706
2         5427792638736934
3         5516102131364383
4         5564768007356822
                ...       
426144    1625149423498289
426145    2135047529277416
426146    3492952121304423
426147    4247709727327181
426148    4507558419857064
Name: stock_id, Length: 426149, dtype: object

In [22]:
df = pd.DataFrame(StockHistory.objects.all().values())
df = df.sort_values('date').reset_index(drop=True)
df = dropna(df)

In [5]:
stocks = Stock.objects.all()
stocks_object = {
    stock.id: stock for stock in stocks
}

In [23]:
df = df.sort_values(['stock_id', 'date'])

In [24]:
df = df.rename(columns={
    'max_price': 'high',
    'min_price': 'low',
    'last_deal_price': 'close',
    'first_price': 'open'
})

In [29]:
df.stock_id

134055       22129017544200
134576       22129017544200
135093       22129017544200
135626       22129017544200
136175       22129017544200
                ...        
393346    71980027636215728
393991    71980027636215728
394567    71980027636215728
395166    71980027636215728
395818    71980027636215728
Name: stock_id, Length: 426058, dtype: int64

In [26]:
df[df.stock_id == 66456062140680464]

,id,stock_id,date,high,low,last_price,close,open,yesterday_price,value,volume,count
300,199325,66456062140680464,2018-07-01,1750.0,1622.0,1646,1623,1750.0,1707,365386196,224482,44
581,199781,66456062140680464,2018-07-02,1728.0,1591.0,1649,1591,1700.0,1646,33182320,19552,9
1049,200268,66456062140680464,2018-07-03,1688.0,1605.0,1649,1686,1610.0,1649,10233518,6247,6
1531,200729,66456062140680464,2018-07-04,1679.0,1590.0,1647,1590,1610.0,1649,21466749,13318,11
1985,201207,66456062140680464,2018-07-07,1590.0,1565.0,1622,1571,1590.0,1647,158807119,101000,11
...,...,...,...,...,...,...,...,...,...,...,...,...
423307,464385,66456062140680464,2021-08-09,13610.0,12810.0,13360,13420,12810.0,12970,19884190850,1488452,672
423794,465163,66456062140680464,2021-08-10,13940.0,12900.0,13360,13380,13940.0,13360,34939936990,2615828,961
424692,465939,66456062140680464,2021-08-11,13680.0,13200.0,13440,13490,13680.0,13360,28202838180,2099178,705
424939,466688,66456062140680464,2021-08-14,13710.0,12770.0,12980,12870,13710.0,13440,18148376410,1397808,684


In [27]:
dfset(map(lambda x: x[0], StockHistory.objects.all().values_list('stock_id')))

{23049019886587905,
 16693610252404739,
 66726992874614788,
 19471788163911687,
 42696242981550091,
 38568786927478796,
 67083516977047564,
 71076372178147339,
 45519261544951819,
 2161110547458064,
 12965822877128721,
 7268401059874834,
 50341528161302545,
 7385624172574740,
 48619517949257749,
 35150770536114195,
 19298748452450329,
 4942127026063388,
 54299437423394846,
 29860265627578401,
 15494954332657697,
 24644999329120295,
 17059960254855208,
 39191249877936167,
 62810990108260391,
 27361263980884014,
 69090868458637360,
 63965059137798192,
 40411537531154482,
 43552974795606067,
 43342306308122676,
 57585821705408565,
 21772258644715569,
 70474983732269112,
 10654052153538617,
 51244286061307960,
 71665987033532474,
 67170215467608124,
 71856634742001725,
 52220424531578944,
 65490886290565185,
 66142616039907394,
 47841327496247362,
 63580313877463104,
 611986653700161,
 50633804639547462,
 3050342257199174,
 21173014653958212,
 68870496728320072,
 62404730109947970,
 673270

In [7]:
def add_cdl_patterns(group):
    try:
        return pd.concat([group, group.ta.cdl_pattern()], axis=1)
    except:
        return pd.DataFrame()

df = df.groupby('stock_id').apply(add_cdl_patterns)

In [8]:
df = df.rename(columns={ 'CDL_DOJI_10_0.1': 'CDL_DOJI' })

In [19]:
res_df = df.drop(['stock_id'], axis=1).reset_index()

In [20]:
res_df

,stock_id,level_1,id,date,high,low,last_price,close,open,yesterday_price,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
0,22129017544200,134055,67778.0,2019-07-28,870.0,783.0,797.0,783.0,870.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22129017544200,134576,68280.0,2019-07-29,764.0,718.0,727.0,722.0,719.0,797.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,22129017544200,135093,68812.0,2019-07-30,769.0,672.0,722.0,743.0,718.0,727.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22129017544200,135626,69324.0,2019-07-31,785.0,728.0,753.0,770.0,731.0,722.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,22129017544200,136175,69871.0,2019-08-03,827.0,770.0,805.0,808.0,800.0,753.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425847,71980027636215728,393346,435103.0,2021-05-25,34300.0,31034.0,31254.0,31034.0,34300.0,32667.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
425848,71980027636215728,393991,435716.0,2021-05-26,30500.0,29692.0,29706.0,29692.0,30500.0,31254.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
425849,71980027636215728,394567,436312.0,2021-05-29,31191.0,28221.0,28225.0,28221.0,31191.0,29706.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
425850,71980027636215728,395166,436939.0,2021-05-30,29400.0,26814.0,26899.0,26814.0,29400.0,28225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
res_df[res_df.stock_id == 66456062140680464]

,stock_id,level_1,id,date,high,low,last_price,close,open,yesterday_price,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
396804,66456062140680464,300,199325.0,2018-07-01,1750.0,1622.0,1646.0,1623.0,1750.0,1707.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
396805,66456062140680464,581,199781.0,2018-07-02,1728.0,1591.0,1649.0,1591.0,1700.0,1646.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
396806,66456062140680464,1049,200268.0,2018-07-03,1688.0,1605.0,1649.0,1686.0,1610.0,1649.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
396807,66456062140680464,1531,200729.0,2018-07-04,1679.0,1590.0,1647.0,1590.0,1610.0,1649.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
396808,66456062140680464,1985,201207.0,2018-07-07,1590.0,1565.0,1622.0,1571.0,1590.0,1647.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397488,66456062140680464,423307,464385.0,2021-08-09,13610.0,12810.0,13360.0,13420.0,12810.0,12970.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
397489,66456062140680464,423794,465163.0,2021-08-10,13940.0,12900.0,13360.0,13380.0,13940.0,13360.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
397490,66456062140680464,424692,465939.0,2021-08-11,13680.0,13200.0,13440.0,13490.0,13680.0,13360.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
397491,66456062140680464,424939,466688.0,2021-08-14,13710.0,12770.0,12980.0,12870.0,13710.0,13440.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
patterns = Pattern.objects.all()

In [15]:
for pattern in tqdm([patterns[0]]):
    df_pattern_key = 'CDL_' + pattern.pattern_key.upper()
    sub_df = df[df[df_pattern_key] != 0].drop(['stock_id'], axis=1).reset_index()[[
        'stock_id', 'date', df_pattern_key
    ]]
    for key, row in tqdm(sub_df.iterrows()):
        try:
            hpap = HistoricalPatternActionPoint(
                stock=stocks_object[row[0]],
                pattern=pattern,
                date=row[1],
                value=row[2]
            )
            hpap.save()
        except Exception as e:
            print(e)
            pass

/var/folders/dr/l8_gry7975l0f3fl8m62sdbh0000gn/T/ipykernel_7794/1656353874.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for pattern in tqdm([patterns[0]]):


  0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/dr/l8_gry7975l0f3fl8m62sdbh0000gn/T/ipykernel_7794/1656353874.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for key, row in tqdm(sub_df.iterrows()):


0it [00:00, ?it/s]

no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_ap

In [14]:
HistoricalPatternActionPoint.objects.all()

<QuerySet []>